# Calculator Applet

Demo that does various calculations - sha, hmac, pbkdf, field math, elliptic curve math

In [29]:
from applets.helpers import get_connection, AppletBase

class Calculator(AppletBase):
    def __init__(self, connection=None):
        super().__init__("B00B5111CC01", connection)

    def sha256(self, d):
        data = bytes([len(d)])+d
        r = "B0A10000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha256(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A20000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def sha512(self, d):
        data = bytes([len(d)])+d
        r = "B0A30000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha512(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A40000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

    def add_mod_FP(self, a, b):
        data = bytes([len(a)])+a+bytes([len(b)])+b
        r = "B0A50000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def add_mod_N(self, a, b):
        data = bytes([len(a)])+a+bytes([len(b)])+b
        r = "B0A60000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

In [30]:
connection = get_connection()
if connection is not None:
    atr=bytes(connection.getATR())
    print(atr.hex())
else:
    print("Failed to open a connection. No cardreaders?")

3bdc18ff8191fe1fc38073c821136605036351000250


In [31]:
app = Calculator(connection)
app.select()

b''

In [32]:
d = b'hello'
k = b'key'

In [33]:
# sha functions from the card
print("sha256:\t\t", app.sha256(d).hex())
print("hmac-sha256:\t", app.hmac_sha256(k, d).hex())
print("sha512:\t\t", app.sha512(d).hex())
print("hmac-sha512:\t", app.hmac_sha512(k, d).hex())

sha256:		 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
hmac-sha256:	 9307b3b915efb5171ff14d8cb55fbcc798c6c0ef1456d66ded1a6aa723a58b7b
sha512:		 9b71d224bd62f3785d96d46ad3ea3d73319bfbc2890caadae2dff72519673ca72323c3d99ba5c11d7c7acc6e14b8c5da0c4663475c2e5c3adef46f73bcdec043
hmac-sha512:	 ff06ab36757777815c008d32c8e14a705b4e7bf310351a06a23b612dc4c7433e7757d20525a5593b71020ea2ee162d2311b247e9855862b270122419652c0c92


In [ ]:
# pbkdf is increadibly slow on the card...
import hashlib, hmac
def pbkdf2_hmac_sha512(password, salt, iterations:int, bytes_to_read:int):
    # xor two arrays
    def binxor(a, b):
        return bytes([x ^ y for (x, y) in zip(a, b)])
    # convert to bytes
    if isinstance(password, str):
        password = password.encode('utf-8')
    if isinstance(salt, str):
        salt = salt.encode('utf-8')
    # result
    r = b''
    # no need in the loop if bytes_to_read <= 64
    # just set i = 1
    for i in range(1,bytes_to_read//64+1+int(bool(bytes_to_read%64))):
        U = hmac.new(password, salt + i.to_bytes(4,'big'), digestmod=hashlib.sha512).digest()
        result = U
        for j in range(2, 1+iterations):
            U = hmac.new(password, U, digestmod=hashlib.sha512).digest()
            result = binxor(result, U)
        r += result
    return r[:bytes_to_read]

In [ ]:
# same using hashlib
print("sha256:\t\t",hashlib.sha256(d).digest().hex())
print("hmac-sha256:\t",hmac.new(k, d, digestmod=hashlib.sha256).digest().hex())
print("sha512:\t\t",hashlib.sha512(d).digest().hex())
print("hmac-sha512:\t",hmac.new(k, d, digestmod=hashlib.sha512).digest().hex())
print("pbkdf2:\t\t", pbkdf2_hmac_sha512(k, d, 2048, 64).hex())

In [ ]:
FP = 2**256 - 2**32 - 977
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

In [ ]:
def to_bytes(n:int):
    return n.to_bytes(32, 'big')

In [ ]:
# random check
import random
for i in range(10):
    a = random.randint(1,N)
    b = random.randint(1,N)
    r = (a+b) % FP
    rr = r.to_bytes(32, 'big').hex()
    rc = app.add_mod_FP(to_bytes(a),to_bytes(b)).hex()
    print(i)
    if (rr!=rc):
        raise RuntimeError("Meh")

In [ ]:
# edge cases - slightly less than mod, slightly more, equal and overflow
a = FP-20
for i, b in enumerate([10,23,20,10500, 1020203124]):
    r = (a+b) % FP
    rr = r.to_bytes(32, 'big').hex()
    rc = app.add_mod_FP(to_bytes(a),to_bytes(b)).hex()
    print(i)
    if (rr!=rc):
        raise RuntimeError("Meh")

In [6]:
from embit import bip32, ec
import os
seed = os.urandom(64)
root = bip32.HDKey.from_seed(seed)
# without prefix, depth, root fingerprint and index
root.serialize()[13:].hex()

'e6a3e77c9b2cea825d0ba1a45a1804f2bbf987543e3de1c1b8eff59df2a5483c00f409fd1817c17ab05c7ed53e031edb68d5dae4c3e1150de55fd8cb0a3885aa2d'

In [7]:
root.child(10, hardened=False).serialize()[13:].hex()

'8b7a74ae7a9ae2c651d96b7af9a6cd609ea02efe6e6c0ececa58adb895c3971a00a0a1412ba9b06e00bae103e06ae3d10ab9dec3c7f6719d7d5a2a4c42cb9a1491'

In [8]:
data = bytes([65])+root.serialize()[13:]+bytes([4,0x00,0,0,10])
r = "B0A90000"+(bytes([len(data)])+data).hex().upper()
app.request(r).hex()


'8b7a74ae7a9ae2c651d96b7af9a6cd609ea02efe6e6c0ececa58adb895c3971a00a0a1412ba9b06e00bae103e06ae3d10ab9dec3c7f6719d7d5a2a4c42cb9a1491'

In [9]:
# point addition
p = root.to_public().key
p.compressed = False
print(p.sec().hex())
pp = ec.PublicKey.parse(p.serialize())
ec.secp256k1.ec_pubkey_tweak_add(pp._point, b'4'*32)
print(pp.sec().hex())

04ed99f09a977dcc5fe18f0677661265e40bb4de07c59cac6121b4f7adfde1409837eadc61d0432cb4104ecb441f950fbde66dbb5ff254b7581407a81716ffc91e
04032c01465c49be8914bbdf8cca3070b9cdd4637afa7ac89ade7d68202c04263f6a2324db048283fed61b69e2a95fae44a4f31c5b20ecad98952df78419a37914


In [10]:
data = bytes([32])+b'4'*32+bytes([65])+p.sec()
r = "B0A70000"+(bytes([len(data)])+data).hex().upper()
app.request(r).hex()


'04032c01465c49be8914bbdf8cca3070b9cdd4637afa7ac89ade7d68202c04263f6a2324db048283fed61b69e2a95fae44a4f31c5b20ecad98952df78419a37914'

In [11]:
p3 = ec.PublicKey.parse(ec.secp256k1.ec_pubkey_serialize(ec.secp256k1.ec_pubkey_combine(p._point,pp._point)))
p3.compressed = False
p3.sec().hex()

'0400c247174702ed370cf95e8c6e372b89bd42e95f43a8fdbb841ec62a1ac19b1d33df058918b50f1c57d8698ea21a8f22940d27de6c3c5a2660a98f67c422e205'

In [12]:
data = bytes([65])+pp.sec()+bytes([65])+p.sec()
r = "B0A80000"+(bytes([len(data)])+data).hex().upper()
app.request(r).hex()


'0400c247174702ed370cf95e8c6e372b89bd42e95f43a8fdbb841ec62a1ac19b1d33df058918b50f1c57d8698ea21a8f22940d27de6c3c5a2660a98f67c422e205'

In [14]:
xpub = root.to_public()
xpub.serialize()[13:].hex()

'e6a3e77c9b2cea825d0ba1a45a1804f2bbf987543e3de1c1b8eff59df2a5483c02ed99f09a977dcc5fe18f0677661265e40bb4de07c59cac6121b4f7adfde14098'

In [25]:
xpub.child(10, hardened=False).serialize()[13:].hex()

'8b7a74ae7a9ae2c651d96b7af9a6cd609ea02efe6e6c0ececa58adb895c3971a024f72834710cf25144a2a5eb18dcb0fd7f36fca86786ebcd519d1e000c1e65fcc'

In [26]:
fullpub = ec.PublicKey.parse(xpub.key.serialize())
fullpub.compressed=False
data = bytes([65])+xpub.serialize()[13:]+bytes([4,0x00,0,0,10])+bytes([65])+fullpub.sec()
r = "B0AA0000"+(bytes([len(data)])+data).hex().upper()
app.request(r).hex()


'8b7a74ae7a9ae2c651d96b7af9a6cd609ea02efe6e6c0ececa58adb895c3971a044f72834710cf25144a2a5eb18dcb0fd7f36fca86786ebcd519d1e000c1e65fcc'

In [44]:
import hashlib
print(hashlib.pbkdf2_hmac('sha512', b'pwd', b'salt', 2048, 64).hex())
print(hashlib.pbkdf2_hmac('sha512', b'pwd', b'salt', 100, 64).hex())

82218bfe339fa8a349423df85eaf89728b4e7dc02360e8c6feec346a8e46232db544f6f2d2fff1430e53f76c0adafd9a2087652b7b943fdc07c0cff4c74252e2
038c39d10762fd77de3a035e2c8c7ba4410f07f8251a28b1d670649abc256e810afb12e6c7e4b3e976fc1be1a387c345b282c9127f02b23862c16859bde9ff7f


In [38]:
import time
time.time()

1586482075.763136

In [40]:
(2048).to_bytes(2, 'big')

b'\x08\x00'

In [45]:
t0 = time.time()
# 2048 iterations -> 0x8000
iterations = 100
data = bytes([2])+iterations.to_bytes(2,'big')+bytes([3])+b'pwd'+bytes([4])+b'salt'
r = "B0A00000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())
dt = time.time()-t0
print(f"{dt} seconds for {iterations} iterations.\nFor 2048 iterations it will be {dt*2048/iterations} seconds")

038c39d10762fd77de3a035e2c8c7ba4410f07f8251a28b1d670649abc256e810afb12e6c7e4b3e976fc1be1a387c345b282c9127f02b23862c16859bde9ff7f
3.6099870204925537 seconds for 100 iterations.
For 2048 iterations it will be 73.9325341796875 seconds


In [46]:
connection.disconnect()